---
title: "Apply myelin labels to basket cell axon"
format: 
    html:
        toc: true 
        code-fold: false
jupyter: python3
---

This example uses the plotting package **PyVista** for interactive rendering. Any rendering program that takes vertices and edges can work.

In [1]:
import pandas as pd
import numpy as np

from caveclient import CAVEclient
import pyvista as pv
import cloudvolume 
import seaborn as sns

## Query a myelinated cell

In [2]:
# Set version to first data release with manual myelination labels
client = CAVEclient("minnie65_public")
client.version = 1078

# Load the myelin values 
myelin_df = client.materialize.tables.vortex_manual_myelination_v0().query(
    desired_resolution=[1,1,1])

# Load the nodes of Ranvier
nodes_df = client.materialize.tables.vortex_manual_nodes_of_ranvier().query(
    desired_resolution=[1,1,1])

Table Owner Notice on vortex_manual_myelination_v0: Myelination status assessed for the axon of the VALID_ID, not the pt_root_id.
Table Owner Notice on vortex_manual_nodes_of_ranvier: Nodes and myelination assessed for the VALID_ID; if pt_root_id does not match valid_id, the segment has undergone proofreading. Proceed with caution.


In [3]:
myelin_df.valid_id.unique()

array([864691135808631069, 864691136210378684, 864691135639556411,
       864691135162983725, 864691136724881917])

In [4]:
# select an example myelinated basket cell
root_id = 864691136210378684 

## Load the static mesh for one cell

In [5]:
# Set the static segmentaiton source to the most recent flat segmentation where that root id is valid
seg_source = 'precomputed://gs://iarpa_microns/minnie/minnie65/seg_m943'

# load from cloudvolume
cv = cloudvolume.CloudVolume(seg_source, progress=False, use_https=True)
mesh = cv.mesh.get(root_id, lod=2)[root_id]

In [6]:
# Render the mesh vertices for pyvista
vertices = mesh.vertices
faces = mesh.faces

# add a column of all 3s to the faces
padded_faces = np.concatenate([np.full((faces.shape[0], 1), 3), faces], axis=1)

mesh_poly = pv.PolyData(vertices, faces=padded_faces)

# Flip with y axis
mesh_poly.points[:, 1] *= -1

In [7]:
# Format the myelin annotations 
myelin_points = np.vstack(myelin_df.query(f"(valid_id=={root_id}) & (tag=='t')").pt_position.to_numpy()).astype(float)
myelin_poly = pv.PolyData(myelin_points)

# Flip with y axis
myelin_poly.points[:, 1] *= -1

In [8]:
# Format the node annotations 
ranvier_points = np.vstack(nodes_df.query(f"(valid_id=={root_id})").pt_position.to_numpy()).astype(float)
ranvier_poly = pv.PolyData(ranvier_points)

# Flip with y axis
ranvier_poly.points[:, 1] *= -1

## Initialize plotting object, add mesh and annotations

In [9]:
pv.set_jupyter_backend("client")
plotter = pv.Plotter(image_scale=10)

silhouette = dict(
    color=[.06, .48, .67],
    line_width=0.5,
)

plotter.add_mesh(mesh_poly, color=[.06, .48, .67], opacity=0.75, silhouette=silhouette)
plotter.add_mesh(myelin_poly, point_size=6, color="orange", opacity=0.5)

plotter.camera_position = 'zy'
plotter.camera.zoom(1.0)
plotter.set_background('#fbfbfb')

plotter.show()

Widget(value='<iframe src="http://localhost:54665/index.html?ui=P_0x1adb63b50d0_0&reconnect=auto" class="pyvis…

In [5]:
#| echo: false
plotter.export_html('../img/interactives/'basket_cell_myelinated.html')


<center>Interactive: Myelination (yellow) annotated along basket cell axon (blue)</center>

<iframe src="../img/interactives/basket_cell_myelinated.html" width="950px" height="600px" style="border: 2px;  "></iframe>